In [1]:
!pip install keras-metrics

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import progressbar
import re
import mne
from scipy import stats


In [107]:
from keras.models import Sequential
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D,MaxPooling1D,Dropout,BatchNormalization,GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras import backend as K
from keras.layers import LeakyReLU
#import keras_metrics as km


In [4]:
import glob
import os

path = r'D:/Datasets/EEG dataset/mental artithematic/data/' # use your path
all_files = glob.glob(os.path.join(path, "*.edf")) 

data1=[]
data2=[]
for filename in (all_files):
    if int(re.findall(r'\d+',filename)[1])==1:
        data1.append(mne.io.read_raw_edf(filename,preload=True).get_data()[0:-3,::]);
    else:
        data2.append(mne.io.read_raw_edf(filename,preload=True).get_data()[0:-3,::]);

Extracting EDF parameters from D:\Datasets\EEG dataset\mental artithematic\data\Subject00_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from D:\Datasets\EEG dataset\mental artithematic\data\Subject00_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Extracting EDF parameters from D:\Datasets\EEG dataset\mental artithematic\data\Subject01_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from D:\Datasets\EEG dataset\mental artithematic\data\Subject01_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Extracting EDF parameters from D:\Datasets\EEG d

Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from D:\Datasets\EEG dataset\mental artithematic\data\Subject17_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Extracting EDF parameters from D:\Datasets\EEG dataset\mental artithematic\data\Subject18_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from D:\Datasets\EEG dataset\mental artithematic\data\Subject18_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Extracting EDF parameters from D:\Datasets\EEG dataset\mental artithematic\data\Subject19_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.i

Extracting EDF parameters from D:\Datasets\EEG dataset\mental artithematic\data\Subject34_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Extracting EDF parameters from D:\Datasets\EEG dataset\mental artithematic\data\Subject35_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from D:\Datasets\EEG dataset\mental artithematic\data\Subject35_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...


In [395]:
data1[0].shape

(18, 91000)

In [6]:
data1x=[]
data2x=[]
for i in range(36):
    data1x.append(stats.zscore(data1[i] ))
for i in range(36):
    data2x.append(stats.zscore(data2[i]))

In [7]:
train1=data1x.copy()
test1 = train1[0:5]
del train1[0:5]
train2=data2x.copy()
test2 = train2[0:5]
del train2[0:5]

In [8]:
print(len(train1))
print(len(test1))
print(len(train2))
print(len(test2))

31
5
31
5


In [396]:
seed=0
sample=500

In [397]:
train_1=[] #add pca after train1 
for i in range(len(train1)):
    train_1.append(np.reshape(train1[i],(-1,sample,18)))
train_2=[]
for i in range(len(train2)):
    train_2.append(np.reshape(train2[i],(-1,sample,18)))

In [398]:
test_1=[]
for i in range(len(test1)):
    test_1.append(np.reshape(test1[i],(-1,sample,18)))
test_2=[]
for i in range(len(test1)):
    test_2.append(np.reshape(test2[i],(-1,sample,18)))    

In [399]:
train_1[0].shape,train_2[0].shape

((182, 500, 18), (62, 500, 18))

In [400]:
data1_train=np.concatenate(train_1,axis=0)
data2_train=np.concatenate(train_2,axis=0)
print(data1_train.shape,data2_train.shape)
X_train=np.concatenate((data1_train,data2_train),axis=0)

(5546, 500, 18) (1922, 500, 18)


In [401]:
data1_test=np.concatenate(test_1,axis=0)
data2_test=np.concatenate(test_2,axis=0)
print(data1_test.shape,data2_test.shape)
X_test=np.concatenate((data1_test,data2_test),axis=0)

(898, 500, 18) (310, 500, 18)


In [402]:
# #3D tensor with shape: (batch, steps, channels)
# X_train=np.swapaxes(X_train,0,1)
# X_test=np.swapaxes(X_test,0,1)
# print(X_train.shape,X_test.shape)

In [403]:
y1=np.zeros(data1_train.shape[0])
y2=np.ones(data2_train.shape[0])
y_train=np.concatenate((y1,y2),axis=0)
y1=np.zeros(data1_test.shape[0])
y2=np.ones(data2_test.shape[0])
y_test=np.concatenate((y1,y2))


In [404]:
X_train.shape,X_test.shape

((7468, 500, 18), (1208, 500, 18))

In [405]:
y_train.shape,y_test.shape

((7468,), (1208,))

In [406]:
idx=np.random.RandomState(seed=seed).permutation(len(y_train))
X_train=X_train[idx]
y_train=y_train[idx]

In [407]:
idx=np.random.RandomState(seed=seed).permutation(len(y_test))
X_test=X_test[idx]
y_test=y_test[idx]

In [408]:
# X_train=np.reshape(X_train,(X_train.shape[0], sample,1))
# X_test=np.reshape(X_test,(X_test.shape[0], sample,1))

# print(X_train.shape,X_test.shape)

In [409]:
y_train=keras.utils.to_categorical(y_train,2)
y_test=keras.utils.to_categorical(y_test,2)

In [410]:
y_test.shape,y_train.shape

((1208, 2), (7468, 2))

In [495]:
model = Sequential()
model.add(Conv1D(16, (8), input_shape=(sample,18)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))




model.add(Conv1D(6, (8)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

#model.add(MaxPooling1D(pool_size=2,strides=2,padding='same'))

#model.add(Conv1D(10, (14), activation='relu',padding='same'))

#model.add(MaxPooling1D(pool_size=2,strides=2,padding='same'))

model.add(Dropout(0.2))

model.add(Flatten())
#model.add(Dense(50, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(2, activation='sigmoid'))


In [496]:
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

In [497]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 493, 16)           2320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 493, 16)           64        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 493, 16)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 486, 6)            774       
_________________________________________________________________
batch_normalization_2 (Batch (None, 486, 6)            24        
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 486, 6)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 486, 6)            0         
__________

In [498]:
# count number of parameters in the model
numParams    = model.count_params()    

# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='checkpoint.h5', verbose=1,
                               save_best_only=True)

In [499]:
class_weights = {0:1, 1:2}

In [ ]:
fittedModel = model.fit(X_train, y_train, batch_size = 100, epochs = 30, 
                        verbose = 2, validation_data=(X_test, y_test),
                        class_weight = class_weights)

Train on 7468 samples, validate on 1208 samples
Epoch 1/30
 - 10s - loss: 0.9301 - acc: 0.6022 - val_loss: 0.6478 - val_acc: 0.6544
Epoch 2/30


In [ ]:
model.history.history.keys()

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(model.history.history['val_acc'],label='val acc')
#plt.plot(model.history.history['acc'],label=' acc')
plt.plot(model.history.history['val_loss'],label='val loss')
#plt.plot(model.history.history['loss'],label=' loss')
# plt.plot(model.history.history['val_precision'],label='val_precision')
# plt.plot(model.history.history['val_recall'],label='val_recall')
plt.title('Accuracy and loss for learning rate 0.0001', fontsize=22)
plt.xlabel('Number of epochs', fontsize=18)
plt.ylabel('Accuracy',fontsize=18)
plt.rc('xtick',labelsize=14)
plt.rc('ytick',labelsize=14)
plt.legend(fontsize=16)
plt.legend(fontsize=16)

In [ ]:
from keras import backend as K 
K.clear_session()

In [ ]:
np.array(model.history.history['val_acc']).mean(),np.array(model.history.history['val_loss']).mean()


In [473]:
# model.load_weights('checkpoint.h5')
# probs       = model.predict(X_test)
# preds       = probs.argmax(axis = -1)  
# acc         = np.mean(preds == y_test.argmax(axis=-1))
# print("Classification accuracy: %f " % (acc))

ValueError: You are trying to load a weight file containing 14 layers into a model with 6 layers.

In [ ]:
# batch_size = 3
# epochs = 200
# model.fit(X_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(X_test,y_test))